# Pivot tables

This notebook is my exercise for understanding pivot tables. It is based in part on Chris Moffitt's tutorial in [PBPYTHON](https://pbpython.com/pandas-pivot-table-explained.html).

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('sales-funnel.xlsx')
df.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


The pandas documentation for pivot tables is [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html).

In [3]:
# Simple pivot table: dataframe, index.
# By default the aggfunc is the average.
pd.pivot_table(df, index=['Manager'], values=['Price']).round(2)

,Price
Manager,
Debra Henley,26111.11
Fred Anderson,35875.00


In [4]:
pd.pivot_table(df, index=['Manager'], values=['Price'], aggfunc='mean').round(2)

,Price
Manager,
Debra Henley,26111.11
Fred Anderson,35875.00


In [5]:
# The index can have multiple levels.
pd.pivot_table(df, index=['Manager', 'Rep'], values=['Price'], aggfunc='mean').round(2)

Price
Manager       Rep                    
Debra Henley  Craig Booker   20000.00
              Daniel Hilton  38333.33
              John Smith     20000.00
Fred Anderson Cedric Moss    27500.00
              Wendy Yule     44250.00

In [6]:
# The aggfunc can have several parameters.
pd.pivot_table(df, index=['Manager', 'Rep'],
               values=['Price'],
               aggfunc=[np.mean, np.sum, len]).round(2)

mean     sum   len
                                Price   Price Price
Manager       Rep                                  
Debra Henley  Craig Booker   20000.00   80000     4
              Daniel Hilton  38333.33  115000     3
              John Smith     20000.00   40000     2
Fred Anderson Cedric Moss    27500.00  110000     4
              Wendy Yule     44250.00  177000     4